In [9]:
import json
import pandas as pd
with open("evaluation_results.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)


df = pd.DataFrame(dataset)
print(f"Total experiments: {len(dataset)}")
df.head()

Total experiments: 1701


,question,answer,contexts,reference,prompt_tokens,completion_tokens,total_tokens,ttft,latency,namespace,top_k,context_precision,context_recall,faithfulness,answer_correctness
0,쿠버네티스가 무엇인가요?,"제공된 문서에서 해당 정보를 찾을 수 없습니다. [문서 1]에는 ""쿠버네티스는:""이...",[쿠버네티스는:],"쿠버네티스는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈...",177,44,469,5.230110,5.299407,cs256-ov0,1,0.0,0.0,0.666667,0.185127
1,쿠버네티스가 무엇인가요?,제공된 문서에서 해당 정보를 찾을 수 없습니다.,[쿠버네티스는:],"쿠버네티스는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈...",177,12,383,4.934436,4.936587,cs256-ov15,1,0.0,0.0,1.000000,0.129893
2,쿠버네티스가 무엇인가요?,"제공된 문서에서 해당 정보를 찾을 수 없습니다.\n\n[문서 1]에는 ""쿠버네티스는...",[쿠버네티스는:],"쿠버네티스는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈...",177,49,459,4.915990,4.918532,cs256-ov30,1,0.0,0.0,1.000000,0.182949
3,쿠버네티스가 무엇인가요?,"[문서 1]에 따르면, 쿠버네티스는 컨테이너화된 워크로드와 서비스를 관리하기 위한 ...",[---\n# reviewers:\n# - bgrant0607\n# - mikeda...,"쿠버네티스는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈...",390,81,919,5.922165,6.194015,cs512-ov0,1,1.0,1.0,1.000000,0.580690
4,쿠버네티스가 무엇인가요?,"문서 1에 따르면, 쿠버네티스에 대한 설명은 다음과 같습니다.\n\n* **명칭...",[<!-- overview -->\n이 페이지에서는 쿠버네티스 개요를 설명한다.\n...,"쿠버네티스는 컨테이너화된 애플리케이션의 배포, 확장 및 관리를 자동화하기 위한 오픈...",380,165,976,6.479982,6.950286,cs512-ov15,1,0.0,0.0,0.888889,0.191891
